In [41]:
import pandas  as pd
import os
import numpy as np
import datetime 
import glob
pd.set_option('display.max_rows', 110)


import dgl
from dgl.data import DGLDataset
import torch
import os
import pandas as pd
import torch
from torch_geometric.utils import dense_to_sparse, to_dense_adj
from scipy.spatial import distance
# Contruct a two-layer GNN model
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

In [52]:
## download dataset from : https://data.epa.gov.tw/dataset/aqx_p_488

df = pd.DataFrame()
for r,ds,fs in os.walk(r'D:\pyData\github\Sally_GNN\data\2020\aqx_p_488_2020-07'):
    fs = [f for f in fs if f.endswith('.csv')]
    for f in fs:
        p = os.path.join(r,f)
        print(p)
        
        df_tmp = pd.read_csv(p)
        
        df = df.append(df_tmp)
        
        del df_tmp
        
df.to_csv(r'./data/df_2021_07.csv',index=False,encoding='ansi')   

D:\pyData\github\Sally_GNN\data\2020\aqx_p_488_2020-07\aqx_p_488_2020-07-01.csv
D:\pyData\github\Sally_GNN\data\2020\aqx_p_488_2020-07\aqx_p_488_2020-07-02.csv
D:\pyData\github\Sally_GNN\data\2020\aqx_p_488_2020-07\aqx_p_488_2020-07-03.csv
D:\pyData\github\Sally_GNN\data\2020\aqx_p_488_2020-07\aqx_p_488_2020-07-04.csv
D:\pyData\github\Sally_GNN\data\2020\aqx_p_488_2020-07\aqx_p_488_2020-07-05.csv
D:\pyData\github\Sally_GNN\data\2020\aqx_p_488_2020-07\aqx_p_488_2020-07-06.csv
D:\pyData\github\Sally_GNN\data\2020\aqx_p_488_2020-07\aqx_p_488_2020-07-07.csv
D:\pyData\github\Sally_GNN\data\2020\aqx_p_488_2020-07\aqx_p_488_2020-07-08.csv
D:\pyData\github\Sally_GNN\data\2020\aqx_p_488_2020-07\aqx_p_488_2020-07-09.csv
D:\pyData\github\Sally_GNN\data\2020\aqx_p_488_2020-07\aqx_p_488_2020-07-10.csv
D:\pyData\github\Sally_GNN\data\2020\aqx_p_488_2020-07\aqx_p_488_2020-07-11.csv
D:\pyData\github\Sally_GNN\data\2020\aqx_p_488_2020-07\aqx_p_488_2020-07-12.csv
D:\pyData\github\Sally_GNN\data\2020\aqx

In [60]:
df = pd.read_csv(r'./data/df_2021_07.csv',encoding='ansi')


# df = df[['SO2', 'CO', 'O3', 'O3_8hr', 'PM10', 'NO2', 'NOx', 'NO', 'WindSpeed', 'WindDirec', 'CO_8hr', 'PM2.5_AVG', 'PM10_AVG', 'SO2_AVG','PM2.5','Longitude','Latitude']]

# df = df.replace({'-':0}).astype(float).dropna()


df[['Longitude','Latitude']] = df[['Longitude','Latitude']].apply(lambda x:round(x,5))


In [62]:
df_g = df.groupby(['SiteName','Longitude','Latitude'])['County'].count()
use_site = [ele[0] for ele in df_g[df_g==744].index]

In [63]:
df.loc[df['SiteName'].isin(use_site)]

,SiteName,County,AQI,Pollutant,Status,SO2,CO,O3,O3_8hr,PM10,...,WindDirec,DataCreationDate,Unit,CO_8hr,PM2.5_AVG,PM10_AVG,SO2_AVG,Longitude,Latitude,SiteId
0,基隆,基隆市,28.0,NaN,良好,2.1,0.26,2.1,2,23.0,...,278,2020-07-01 06:00,NaN,0.2,8.0,26.0,2.0,121.76006,25.12917,1
1,汐止,新北市,38.0,NaN,良好,4.2,0.37,4.6,3,35.0,...,243,2020-07-01 06:00,NaN,0.3,12.0,29.0,3.0,121.64230,25.06713,2
2,萬里,新北市,31.0,NaN,良好,2.0,0.2,4.7,6,19.0,...,220,2020-07-01 06:00,NaN,0.2,10.0,20.0,3.0,121.68988,25.17967,3
3,新店,新北市,37.0,NaN,良好,1.4,0.31,6.7,13,27.0,...,192,2020-07-01 06:00,NaN,0.2,11.0,24.0,2.0,121.53778,24.97722,4
4,觀音,桃園市,14.0,NaN,良好,0.5,0.11,17,15,12.0,...,234,2020-07-01 06:00,NaN,0.1,3.0,10.0,1.0,121.08276,25.03550,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59970,淡水,新北市,16.0,NaN,良好,1.7,0.05,11.3,13,5,...,130,2020-07-31 00:00,NaN,0.1,5.0,6,2.0,121.44924,25.16450,10
59971,士林,臺北市,11.0,NaN,良好,1.7,0.05,11.9,12,5,...,95,2020-07-31 00:00,NaN,0.1,3.0,5,2.0,121.51539,25.10542,11
59972,中山,臺北市,8.0,NaN,良好,2.2,0.22,6.6,4,5,...,110,2020-07-31 00:00,NaN,0.4,1.0,9,2.0,121.52653,25.06236,12
59973,萬華,臺北市,10.0,NaN,良好,1.7,0.14,9.1,6,10,...,86,2020-07-31 00:00,NaN,0.3,3.0,7,2.0,121.50797,25.04650,13


In [137]:
df = df.replace({"-":0})

In [138]:
dt_len =0 
dt_ary = np.sort(df['DataCreationDate'].unique())
dt_len = len(dt_ary)

In [139]:
site_loc_dict = {r['SiteName']:(r['Longitude'],r['Latitude']) for i,r in df.loc[df['SiteName'].isin(use_site),['SiteName','Longitude','Latitude']].drop_duplicates().iterrows()}
site_loc_dict = dict(sorted(site_loc_dict.items(), key=lambda item: item[1][0],reverse=True))
site_ary = np.array(list(site_loc_dict.keys()))
site_len = len(site_ary)

In [140]:
feat_ary = [ 'SO2', 'CO', 'O3', 'O3_8hr', 'PM10', 'NO2', 'NOx', 'NO', 'WindSpeed', 'WindDirec', 'CO_8hr', 'PM2.5_AVG', 'PM10_AVG', 'SO2_AVG','PM2.5']
feat_len = len(feat_ary)

In [183]:
df[feat_ary].head()

,SO2,CO,O3,O3_8hr,PM10,NO2,NOx,NO,WindSpeed,WindDirec,CO_8hr,PM2.5_AVG,PM10_AVG,SO2_AVG,PM2.5
0,2.1,0.26,2.1,2,23.0,8.2,20.7,12.5,0.6,278,0.2,8.0,26.0,2.0,8.0
1,4.2,0.37,4.6,3,35.0,10.8,26.5,15.7,0.8,243,0.3,12.0,29.0,3.0,10.0
2,2.0,0.2,4.7,6,19.0,7.0,8.0,1.1,3.2,220,0.2,10.0,20.0,3.0,8.0
3,1.4,0.31,6.7,13,27.0,6.2,6.7,0.5,1.4,192,0.2,11.0,24.0,2.0,12.0
4,0.5,0.11,17,15,12.0,4.0,4.2,0.3,3.7,234,0.1,3.0,10.0,1.0,2.0


In [141]:
ary = np.zeros((dt_len,site_len,feat_len))
ary.shape

(744, 80, 15)

In [142]:
_SiteName = site_ary[0]
_Date = dt_ary[0]



for i,_Date in enumerate(dt_ary) :
        print(_Date)
    for j,_SiteName in enumerate(site_ary):

            cond1 = (df['SiteName']==_SiteName)
            cond2 = (df['DataCreationDate']==_Date)            
            
            ary[i,j,:] = df.loc[cond1 & cond2 ,  list(feat_ary)].values


冬山 2020-07-01 00:00
基隆 2020-07-01 00:00
宜蘭 2020-07-01 00:00
萬里 2020-07-01 00:00
汐止 2020-07-01 00:00
花蓮 2020-07-01 00:00
松山 2020-07-01 00:00
富貴角 2020-07-01 00:00
新店 2020-07-01 00:00
陽明 2020-07-01 00:00
古亭 2020-07-01 00:00
中山 2020-07-01 00:00
永和 2020-07-01 00:00
士林 2020-07-01 00:00
大同 2020-07-01 00:00
萬華 2020-07-01 00:00
三重 2020-07-01 00:00
菜寮 2020-07-01 00:00
板橋 2020-07-01 00:00
土城 2020-07-01 00:00
淡水 2020-07-01 00:00
新莊 2020-07-01 00:00
林口 2020-07-01 00:00
桃園 2020-07-01 00:00
中壢 2020-07-01 00:00
龍潭 2020-07-01 00:00
平鎮 2020-07-01 00:00
大園 2020-07-01 00:00
關山 2020-07-01 00:00
臺東 2020-07-01 00:00
竹東 2020-07-01 00:00
觀音 2020-07-01 00:00
湖口 2020-07-01 00:00
新竹 2020-07-01 00:00
埔里 2020-07-01 00:00
頭份 2020-07-01 00:00
苗栗 2020-07-01 00:00
恆春 2020-07-01 00:00
三義 2020-07-01 00:00
豐原 2020-07-01 00:00
南投 2020-07-01 00:00
大里 2020-07-01 00:00
竹山 2020-07-01 00:00
忠明 2020-07-01 00:00
西屯 2020-07-01 00:00
沙鹿 2020-07-01 00:00
潮州 2020-07-01 00:00
斗六 2020-07-01 00:00
彰化 2020-07-01 00:00
美濃 2020-07-01 00:00

In [160]:
custom_dict = {s:i for i,s in enumerate(site_ary)}

for i,_Date in enumerate(dt_ary) :
        cond1 = (df['SiteName'].isin(site_ary))
        cond2 = (df['DataCreationDate']==_Date)  

        cond = cond1 & cond2
        ary[i,:,:] = df.loc[cond ,:].iloc[df.loc[cond ,  :]['SiteName'].map(custom_dict).argsort()][feat_ary].values

2020-07-01 00:00
2020-07-01 01:00
2020-07-01 02:00
2020-07-01 03:00
2020-07-01 04:00
2020-07-01 05:00
2020-07-01 06:00
2020-07-01 07:00
2020-07-01 08:00
2020-07-01 09:00
2020-07-01 10:00
2020-07-01 11:00
2020-07-01 12:00
2020-07-01 13:00
2020-07-01 14:00
2020-07-01 15:00
2020-07-01 16:00
2020-07-01 17:00
2020-07-01 18:00
2020-07-01 19:00
2020-07-01 20:00
2020-07-01 21:00
2020-07-01 22:00
2020-07-01 23:00
2020-07-02 00:00
2020-07-02 01:00
2020-07-02 02:00
2020-07-02 03:00
2020-07-02 04:00
2020-07-02 05:00
2020-07-02 06:00
2020-07-02 07:00
2020-07-02 08:00
2020-07-02 09:00
2020-07-02 10:00
2020-07-02 11:00
2020-07-02 12:00
2020-07-02 13:00
2020-07-02 14:00
2020-07-02 15:00
2020-07-02 16:00
2020-07-02 17:00
2020-07-02 18:00
2020-07-02 19:00
2020-07-02 20:00
2020-07-02 21:00
2020-07-02 22:00
2020-07-02 23:00
2020-07-03 00:00
2020-07-03 01:00
2020-07-03 02:00
2020-07-03 03:00
2020-07-03 04:00
2020-07-03 05:00
2020-07-03 06:00
2020-07-03 07:00
2020-07-03 08:00
2020-07-03 09:00
2020-07-03 10:

In [162]:
ary.shape # datetime, stie, features 

(744, 80, 15)

In [179]:
# https://github.com/google-research/social_cascades
from torch.nn.parameter import Parameter
import math 
class RNN(nn.Module):
    """RNN model."""

    def __init__(self, in_dim, hid_dim, num_label, num_layers=1, dropout=0.7):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size=in_dim, hidden_size=hid_dim,
                          num_layers=num_layers, dropout=dropout)
        self.linear = nn.Linear(hid_dim, num_label)

    def forward(self, x):
        _, hid_state = self.rnn(x)
        last_hidden_out = hid_state[-1].squeeze()
        return self.linear(last_hidden_out)


class GraphConvolution(nn.Module):
    """A graph convolution layer.
    It follows https://github.com/tkipf/pygcn and
    https://arxiv.org/abs/1609.02907.
    Attributes:
    in_dim: An integer indicating the dimension of input.
    out_dim: An integer indicating the dimension of output.
    weight: Torch tensor with dimension (in_dim, out_dim).
    bias: Torch tensor with dimension out_dim.
    """

    def __init__(self, in_dim, out_dim, bias=True):
        """Initializes the instance.
        Args:
          in_dim: An integer indicating the dimension of input.
          out_dim: An integer indicating the dimension of output.
          bias: An optional boolean variable indicating whether to have bias.
        """
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.weight = Parameter(torch.FloatTensor(in_dim, out_dim))
        self.bias = Parameter(torch.FloatTensor(out_dim)) if bias else None
        self.reset_parameters()

    def reset_parameters(self):
        """Initializes parameters following the default pytorch initialization."""
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x_feature, adjacency_matrix):
        """Forward function.
        Args:
        x_feature: A tensor of input features.
        adjacency_matrix: A tensor of adjacency matrix.
        Returns:
        A tensor of output embedding.
        """
        support = torch.mm(x_feature, self.weight)
        output = torch.spmm(adjacency_matrix, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

        
class GCN(nn.Module):
    """A 2-layer GCN model.
    It follows https://github.com/tkipf/pygcn and
    https://arxiv.org/abs/1609.02907.
    Attributes:
    gc1: A gcn layer.
    gc2: A gcn layer.
    dropout: A float indicating the dropout rate.
    """

    def __init__(self, in_dim, hid_dim, out_dim, dropout=0.0, bias=True):
        """Initializes the instance.
        Args:
          in_dim: An integer indicating the dimension of input.
          hid_dim: An integer indicating the dimension of hidden layer.
          out_dim: An integer indicating the dimension of output.
          dropout: An optional float variable indicating the dropout rate.
          bias: An optional variable indicating whether to include bias.
        """
        super().__init__()
        self.gc1 = GraphConvolution(in_dim, hid_dim, bias=bias)
        self.gc2 = GraphConvolution(hid_dim, out_dim, bias=bias)
        self.dropout = dropout

    def forward(self, x_feature, adjacency_matrix):
        """Forward function.
        This function applys two GCN layers with dropout.
        Args:
          x_feature: A tensor of input features of shape (number of nodes,
            feature dimension).
          adjacency_matrix: A tensor of adjacent matrix.
        Returns:
          A tensor of output embedding of shape (number of nodes, output dimension).
        """
        # replace relu by elu
        x_feature = nn.functional.elu(self.gc1(x_feature, adjacency_matrix))
        x_feature = nn.functional.dropout(x_feature, self.dropout,
                                          training=self.training)
        x_feature = self.gc2(x_feature, adjacency_matrix)
        return x_feature

    
    


class GCNRNN(nn.Module):
    """A GCN+RNN post sequence model.
    Attributes:
    gcn: A GCN class.
    rnn: A RNN or LSTM class.
    """

    def __init__(self, gcn_in_dim, gcn_hid_dim, gcn_out_dim, rnn_type,
           rnn_hid_dim, rnn_out_dim, rnn_num_layers=1, rnn_dropout=0.0,
           bi_direct=False, gcn_dropout=0.0, gcn_bias=True):
        """Initializes the instance.
        Args:
        gcn_in_dim: (integer) Dimension of GCN input.
        gcn_hid_dim: (integer) Dimension of GCN hidden layer.
        gcn_out_dim: (integer) Dimension of GCN output.
        rnn_type: (string, 'RNN' or 'LSTM') Whether to use RNN or LSTM.
        rnn_hid_dim: (integer) Dimension of RNN/LSTM hidden layer.
        rnn_out_dim: (integer) Dimension of RNN/LSTM output.
        rnn_num_layers: (integer, default=1) Number of layers in RNN/LSTM.
        rnn_dropout: (float in [0.0, 1.0], default=0.0) Dropout rate in RNN/LSTM.
        bi_direct: (boolean, default=False) Whether RNN/LSTM is bidirectional.
        gcn_dropout: (float in [0.0, 1.0], default=0.0) Dropout rate in GCN.
        gcn_bias: (boolean, default=True) Whether to include bias in GCN.
        """
        super().__init__()
        self.gcn = GCN(in_dim=gcn_in_dim, hid_dim=gcn_hid_dim, out_dim=gcn_out_dim,
                   dropout=gcn_dropout, bias=gcn_bias)
        if rnn_type == 'RNN':
            self.rnn = RNN(in_dim=gcn_out_dim, hid_dim=rnn_hid_dim,
                         num_label=rnn_out_dim, num_layers=rnn_num_layers,
                         dropout=rnn_dropout)
        elif rnn_type == 'LSTM':
            self.rnn = LSTM(in_dim=gcn_out_dim, hid_dim=rnn_hid_dim,
                          num_label=rnn_out_dim, num_layers=rnn_num_layers,
                          dropout=rnn_dropout, bi_direct=bi_direct)

    def forward(self, batch_x):
        """Forward function.
        Args:
          batch_x: A batch of input data.
        Returns:
          A batch tensor of output embedding.
        """
        batch_x_post_embedding = []
        for x_sequence in batch_x:
            post_embeddings = torch.stack(
              [self.gcn(x_feature, adj)[0] for x_feature, adj in x_sequence])
            batch_x_post_embedding.append(post_embeddings)
        batch_x_post_embedding = utils.data_padding(batch_x_post_embedding)
        output = self.rnn(batch_x_post_embedding)
        return output

In [ ]:
class RNN(nn.Module):
    """RNN model."""

    def __init__(self, in_dim, hid_dim, num_label, num_layers=1, dropout=0.1):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size=in_dim, hidden_size=hid_dim,
                          num_layers=num_layers, dropout=dropout)
        self.linear = nn.Linear(hid_dim, num_label)

    def forward(self, x):
        _, hid_state = self.rnn(x)
        last_hidden_out = hid_state[-1].squeeze()
        return self.linear(last_hidden_out)


class SAGE_RNN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv( in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv( in_feats=hid_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv3 = dglnn.SAGEConv( in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')
        
        self.rnn = RNN(in_dim=gcn_out_dim, hid_dim=rnn_hid_dim, num_label=rnn_out_dim, num_layers=1, dropout=0.1)
        
        self.linear1 = nn.Linear(out_feats,1)
        
    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        h = F.relu(h)
        h = self.conv3(graph, h)
        
        
    def forward(self, batch_x):
        """Forward function.
        Args:
          batch_x: A batch of input data.
        Returns:
          A batch tensor of output embedding.
        """
        batch_x_post_embedding = []
        for x_sequence in batch_x:
            post_embeddings = torch.stack(
              [self.gcn(x_feature, adj)[0] for x_feature, adj in x_sequence])
            batch_x_post_embedding.append(post_embeddings)
        batch_x_post_embedding = utils.data_padding(batch_x_post_embedding)
        output = self.rnn(batch_x_post_embedding)
        return output

In [181]:
gcn_rnn_model = GCNRNN(  gcn_in_dim=64, 
                         gcn_hid_dim=16,
                         gcn_out_dim=16, 
                         rnn_type='RNN',
                         rnn_hid_dim=16, 
                         rnn_out_dim=1,
                         rnn_num_layers=1, 
                         rnn_dropout=0.1,
                         bi_direct=False, 
                         gcn_dropout=0.1,
                         gcn_bias=True)#.to(device)

In [ ]:
gcn_rnn_model